In [1]:
import pandas as pd
import numpy as np

In [6]:
# 행정동 데이터 불러오기
merged_df = pd.read_csv('merged.csv', encoding='utf-8')
merged_df.head()

,행정동,1218_유동인구,1218_거주인구,1218_유동인구/거주인구,2103_유동인구,2103_거주인구,2103_유동인구/거주인구,카메라대수,면적대비_CCTV,보정_범죄자수,보정_면적대비범죄자수,유흥매출액,유흥+일반매출액,여성1인가구수
0,중앙동,2.410229e+06,5033,478.885184,575172.600,5033,114.280270,42,33.070866,1.0,0.787402,8.151190e+08,53751417511,490
1,효동,1.387323e+06,26167,53.018043,447480.467,26167,17.100946,210,87.136929,0.1,0.041494,1.979914e+09,57881696036,936
2,신인동,1.152293e+06,12549,91.823481,453473.133,12549,36.136197,42,50.000000,1.0,1.190476,3.350896e+06,2839351265,528
3,판암1동,1.012310e+06,12807,79.043476,313242.867,12807,24.458723,61,11.359404,1.0,0.186220,8.708482e+07,13905259332,640
4,판암2동,2.776619e+05,8877,31.278803,95004.600,8877,10.702332,30,39.473684,3.0,3.947368,9.093965e+07,1449230531,1313


In [7]:
# 각 행정동별과 데이터 지표에 대한 dataframe 생성
idx = []  # 행정동
for dong in merged_df['행정동']:
    idx.append(dong)

# 지표
factors = ['1218_유동인구/거주인구', '2103_유동인구/거주인구', '면적대비_CCTV', '보정_면적대비범죄자수', '유흥매출액', '여성1인가구수']

# init dataframe
weight_df = pd.DataFrame(columns=factors, index=idx)
weight_df.head()

,1218_유동인구/거주인구,2103_유동인구/거주인구,면적대비_CCTV,보정_면적대비범죄자수,유흥매출액,여성1인가구수
중앙동,NaN,NaN,NaN,NaN,NaN,NaN
효동,NaN,NaN,NaN,NaN,NaN,NaN
신인동,NaN,NaN,NaN,NaN,NaN,NaN
판암1동,NaN,NaN,NaN,NaN,NaN,NaN
판암2동,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# 각 데이터 불러온 후 weight_df에 입력 (6번 반복)
data = pd.read_csv('weights/weight_여성1인가구수.csv', encoding='utf-8')
weight = data['weight'].tolist()
weight_df['여성1인가구수'] = weight

In [22]:
weight_df.head()

,1218_유동인구/거주인구,2103_유동인구/거주인구,면적대비_CCTV,보정_면적대비범죄자수,유흥매출액,여성1인가구수
중앙동,0.063476,0.040399,0.012698,0.013465,0.007244,0.005965
효동,0.007027,0.006045,0.004893,0.000710,0.017595,0.011395
신인동,0.012171,0.012774,0.010254,0.020358,0.000030,0.006428
판암1동,0.010477,0.008646,0.015832,0.003184,0.000774,0.007791
판암2동,0.004146,0.003783,0.011774,0.067503,0.000808,0.015984


In [26]:
# 데이터 지표 가중치 불러오기
factor_df = pd.read_csv('weight_factor.csv', encoding='utf-8')
factor_df

,factor,sum,weight
0,1218_유동인구/거주인구,41.777778,0.094682
1,2103_유동인구/거주인구,125.333333,0.284045
2,면적대비_CCTV,16.444444,0.037268
3,보정_면적대비범죄자수,28.622222,0.064867
4,유흥매출액,141.600000,0.320911
5,여성1인가구수,87.466667,0.198227


In [27]:
# 각 행정동별 최종 합산 위험도 계산
def calculate(x):
    r = x['1218_유동인구/거주인구'] * 0.094682 + x['2103_유동인구/거주인구'] * 0.284045 + x['면적대비_CCTV'] * 0.037268 + x['보정_면적대비범죄자수'] *  0.064867 + x['유흥매출액'] * 0.320911 + x['여성1인가구수'] * 0.198227
    return r

In [30]:
weight_df['result'] = weight_df.apply(lambda x: calculate(x), axis=1)
weight_df.head()

,1218_유동인구/거주인구,2103_유동인구/거주인구,면적대비_CCTV,보정_면적대비범죄자수,유흥매출액,여성1인가구수,result
중앙동,0.063476,0.040399,0.012698,0.013465,0.007244,0.005965,0.022339
효동,0.007027,0.006045,0.004893,0.000710,0.017595,0.011395,0.010516
신인동,0.012171,0.012774,0.010254,0.020358,0.000030,0.006428,0.007767
판암1동,0.010477,0.008646,0.015832,0.003184,0.000774,0.007791,0.006037
판암2동,0.004146,0.003783,0.011774,0.067503,0.000808,0.015984,0.009712


In [32]:
# 모든 위험도의 합 = 1
total = weight_df['result'].sum()
total

1.0

In [33]:
# 최종 위험도가 높은 순서로 행정동 나열
danger = weight_df[['result']]
danger.sort_values(by = 'result', ascending=False, inplace=True)

<ipython-input-33-7172d1bf35e3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  danger.sort_values(by = 'result', ascending=False, inplace=True)


In [34]:
danger.head()

,result
온천1동,0.079202
탄방동,0.031905
둔산2동,0.031487
대흥동,0.028446
유천1동,0.024056


In [35]:
danger.to_csv('위험도.csv')